# Ferramentas para NLP

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.functions import regexp_replace, lower, monotonically_increasing_id, lit

In [4]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [56]:
df = spark.read.option("encoding", "ISO-8859-1").text(['data.txt', 'class.txt', 'newtest.txt'])

In [57]:
df = df.withColumn("value", regexp_replace(lower(df["value"]), "[$&+,:;=?@#|'<>.-^*()%!]", ""))

In [58]:
df = df.select('*').withColumn("id", monotonically_increasing_id())

In [59]:
df.show()

+--------------------+-----------+
|               value|         id|
+--------------------+-----------+
|classificacao reg...|          0|
|aqui um arquivo d...| 8589934592|
|    novão pra checar|17179869184|
+--------------------+-----------+



## RegexTokenizer

O RegexTokenizer permite uma tokenização mais avançada com base na correspondência de expressões regulares (regex). Por padrão, o parâmetro “pattern” (regex, padrão: "\ s +") é usado como delimitadores para dividir o texto de entrada. Alternativamente, os usuários podem definir o parâmetro “gaps” para false, indicando que o “padrão” da regex denota “tokens”, em vez de dividir lacunas, e encontrar todas as ocorrências correspondentes como o resultado da tokenização.

In [60]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [61]:
tokenizer = Tokenizer(inputCol="value", outputCol="words")

#extrai o token de acordo com o que foi especificado - aqui foi especificado
regexTokenizer = RegexTokenizer(inputCol="value", outputCol="words", pattern="\\W")
# pattern="\\w+", gaps(False)

countTokens = udf(lambda words: len(words), IntegerType())

tokenized = tokenizer.transform(df)
tokenized.toPandas()


,value,id,words
0,classificacao regressao modelos sao legais,0,"[classificacao, regressao, modelos, sao, legais]"
1,aqui um arquivo de texto,8589934592,"[aqui, um, arquivo, de, texto]"
2,novão pra checar,17179869184,"[novão, pra, checar]"


In [62]:
tokenized.select("value", "words")\
    .withColumn("tokens", countTokens(col("words"))).toPandas()


,value,words,tokens
0,classificacao regressao modelos sao legais,"[classificacao, regressao, modelos, sao, legais]",5
1,aqui um arquivo de texto,"[aqui, um, arquivo, de, texto]",5
2,novão pra checar,"[novão, pra, checar]",3



## Removendo Stop Words

Stop Words são palavras que devem ser excluídas da entrada, normalmente porque as palavras aparecem com frequência e não têm tanto significado.

StopWordsRemover toma como entrada uma sequência de strings (por exemplo, a saída de um Tokenizer) e elimina todas as palavras de parada das seqüências de entrada. A lista de Stop Words é especificada pelo parâmetro stopWords. As Stop Words padrões para alguns idiomas são acessíveis chamando StopWordsRemover.loadDefaultStopWords(idioma), para o qual as opções disponíveis são “danish”, “dutch”, “english”, “finnish”, “french”, “german”, “hungarian”, “italian”, “norwegian”, “portuguese”, “russian”, “spanish”, “swedish” and “turkish”. Um parâmetro booleano caseSensitive indica se as correspondências devem diferenciar maiúsculas de minúsculas (falso por padrão).

In [63]:
from pyspark.ml.feature import StopWordsRemover
language="portuguese"
remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords=StopWordsRemover.loadDefaultStopWords(language))
remover = remover.transform(tokenized)

In [64]:
remover.toPandas()

,value,id,words,filtered
0,classificacao regressao modelos sao legais,0,"[classificacao, regressao, modelos, sao, legais]","[classificacao, regressao, modelos, sao, legais]"
1,aqui um arquivo de texto,8589934592,"[aqui, um, arquivo, de, texto]","[aqui, arquivo, texto]"
2,novão pra checar,17179869184,"[novão, pra, checar]","[novão, pra, checar]"


## N-Gramas

NGram toma como entrada uma seqüência strings (por exemplo, a saída de um Tokenizer). O parâmetro n é usado para determinar o número de termos em cada n-grama. A saída consistirá de uma sequência de n-gramas, onde cada n-grama é representado por uma string delimitada por espaço de n palavras consecutivas. Se a sequência de entrada contiver menos de n cadeias, nenhuma saída será produzida.


In [65]:
from pyspark.ml.feature import NGram

ngram = NGram(n=2, inputCol="filtered", outputCol="ngrams")
ngramDataFrame = ngram.transform(remover)

ngramDataFrame.select("ngrams").show(truncate=False)

+---------------------------------------------------------------------+
|ngrams                                                               |
+---------------------------------------------------------------------+
|[classificacao regressao, regressao modelos, modelos sao, sao legais]|
|[aqui arquivo, arquivo texto]                                        |
|[novão pra, pra checar]                                              |
+---------------------------------------------------------------------+



_______
# TF-IDF
_______

In [11]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sentenceData = spark.createDataFrame([
    (0.0, "Ola ja ouvi falar de Spark"),
    (1.0, "Eu gostaria de usar datascience em tudo"),
    (2.0, "Classificacao,regressao,modelos,sao,legais")
], ["label", "sentence"])

sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Ola ja ouvi falar...|
|  1.0|Eu gostaria de us...|
|  2.0|Classificacao,reg...|
+-----+--------------------+



In [12]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
wordsData.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Ola ja ouvi falar...|[ola, ja, ouvi, f...|
|  1.0|Eu gostaria de us...|[eu, gostaria, de...|
|  2.0|Classificacao,reg...|[classificacao,re...|
+-----+--------------------+--------------------+



## CountVectorizer

O CountVectorizer selecionará as melhores palavras com um tamanho igual ao valor especificado pelo parâmetro vocabSize, ordenadas pela frequência do termo em todo o corpus. Um parâmetro opcional minDF também afeta o processo de ajuste, especificando o número mínimo dos documentos em que um termo deve aparecer para ser incluído no vocabulário. 

In [ ]:
from pyspark.ml.feature import CountVectorizer

In [ ]:
def count_vectorizer(data, inputName="words"):
    cv = CountVectorizer(inputCol=inputName, outputCol="features")
    #minDf o termo só aparece se estiver em mais documentos que o minimo.
    model = cv.fit(df)
    result = model.transform(df)
    return result, model.vocabulary

In [67]:


#cada linha é um "bag of words" com um ID.
# df = spark.createDataFrame([
#     (0, "a b c j".split(" ")),
#     (1, ["a", "b", "b", "c", "a", "k"])
# ], ["id", "words"])

cv = CountVectorizer(inputCol="ngrams", outputCol="features")
#minDf o termo só aparece se estiver em mais documentos que o minimo.
model = cv.fit(ngramDataFrame)

result = model.transform(ngramDataFrame)
result.toPandas()

,value,id,words,filtered,ngrams,features
0,classificacao regressao modelos sao legais,0,"[classificacao, regressao, modelos, sao, legais]","[classificacao, regressao, modelos, sao, legais]","[classificacao regressao, regressao modelos, m...","(1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0)"
1,aqui um arquivo de texto,8589934592,"[aqui, um, arquivo, de, texto]","[aqui, arquivo, texto]","[aqui arquivo, arquivo texto]","(0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
2,novão pra checar,17179869184,"[novão, pra, checar]","[novão, pra, checar]","[novão pra, pra checar]","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0)"


In [68]:
result.select('ngrams').collect()[0][0]

['classificacao regressao', 'regressao modelos', 'modelos sao', 'sao legais']

In [73]:
result.select('features').collect()[1][0].toArray()

array([0., 1., 0., 0., 1., 0., 0., 0.])

In [70]:
model.vocabulary

['classificacao regressao',
 'arquivo texto',
 'sao legais',
 'modelos sao',
 'aqui arquivo',
 'pra checar',
 'novão pra',
 'regressao modelos']